In [16]:
import torch
import torch_geometric.nn as nn

In [63]:
class CustomLayer(nn.MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).
        
        self.linear = nn.Linear(in_channels, out_channels, bias=False)
        self.bias = torch.nn.Parameter(torch.Tensor(out_channels))
        
        self.reset_parameters()
        
    def forward(self):
        pass
        
    def message(self):
        pass
        
    def reset_parameters(self):
        pass
        

In [67]:
class GNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.customLayer = CustomLayer(3, 1)

In [68]:
model = GNN()